In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from api_keys import g_key

gmaps.configure(api_key=g_key)


In [2]:
military_bases = "military-bases.xlsm"
bases_df = pd.read_excel("military-bases.xlsm")
bases_df.rename(columns = {'Geo Point':'geo_point'}, inplace = True)
bases_df.rename(columns = {'Site Name':'site_name'}, inplace = True)
bases_df.head()

,geo_point,OBJECTID,COMPONENT,site_name,State Terr,COUNTRY,Oper Stat
0,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Guam,Guam,Active
1,"42.2096141173, -87.8061179586",485,Navy Active,Fort Sheridan,Illinois,United States,Active
2,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Guam,Guam,Active
3,"34.6687857751, -99.2708984387",528,AF Active,Altus AFB,Oklahoma,United States,Active
4,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Virginia,United States,Active


In [3]:
bases_df['geo_point'] = bases_df['geo_point'].astype(str)

In [4]:
bases_df.dtypes

geo_point     object
OBJECTID       int64
COMPONENT     object
site_name     object
State Terr    object
COUNTRY       object
Oper Stat     object
dtype: object

In [5]:
new_bases_df = bases_df["geo_point"].str.split(",", n=1, expand = True)
bases_df["lat"]= new_bases_df[0]
bases_df["long"]= new_bases_df[1]
bases_df.drop(columns =["geo_point"], inplace = True)
bases_df.head()

,OBJECTID,COMPONENT,site_name,State Terr,COUNTRY,Oper Stat,lat,long
0,239,Navy Active,Tumon Tank Farm,Guam,Guam,Active,13.5179166878,144.811096965
1,485,Navy Active,Fort Sheridan,Illinois,United States,Active,42.2096141173,-87.8061179586
2,240,Navy Active,Potts Tank Farm,Guam,Guam,Active,13.5867134229,144.862326053
3,528,AF Active,Altus AFB,Oklahoma,United States,Active,34.6687857751,-99.2708984387
4,718,Army Active,Fort Myer,Virginia,United States,Active,38.8786170847,-77.075713083


In [6]:
bases_df[["lat","long"]] = bases_df[["lat","long"]].astype(float)
bases_df.dtypes

OBJECTID        int64
COMPONENT      object
site_name      object
State Terr     object
COUNTRY        object
Oper Stat      object
lat           float64
long          float64
dtype: object

In [7]:
ufo_csv = "ufo_sighting_data.csv"
ufo_df = pd.read_csv("ufo_sighting_data.csv")
ufo_df = ufo_df.dropna(how='any')
ufo_df['latitude'] = pd.to_numeric(ufo_df['latitude'])
ufo_df['length_of_encounter_seconds'] = pd.to_numeric(ufo_df['length_of_encounter_seconds'])

ufo_df.head()

C:\Users\kdsan\terminal_prework\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date_time,city,state/province,country,UFO_shape,length_of_encounter_seconds,described_duration_of_encounter,description,date_documented,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889
7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333


In [8]:
simplified = ufo_df[["city", "latitude", "longitude","Date_time"]]
simplified.sort_values(by=["city"])
simplified.drop_duplicates(["city"], keep='last')
simplified.head()


,city,latitude,longitude,Date_time
0,san marcos,29.883056,-97.941111,10/10/1949 20:30
3,edna,28.978333,-96.645833,10/10/1956 21:00
4,kaneohe,21.418056,-157.803611,10/10/1960 20:00
5,bristol,36.595000,-82.188889,10/10/1961 19:00
7,norwalk,41.117500,-73.408333,10/10/1965 23:45


In [9]:
city_stacked = ufo_df.groupby(['city']).agg({'Date_time': 'count'})
city_stacked.rename(columns = {'Date_time':'frequency'}, inplace = True)
city_stacked

,frequency
city,
abbeville,6
abbeville (lake secession),1
abbotsford (canada),14
aberdeen,17
abilene,32
...,...
zion grove,1
zionsville,7
zortman,2


In [10]:
merge_table = pd.merge(city_stacked, simplified, on="city", how="left")
ranked_df = merge_table.nlargest(70000, 'frequency')
ranked_df
ranked_df.drop_duplicates(subset = "city", keep= "last", inplace = True)

In [11]:
locations = ranked_df[["latitude","longitude"]]

frequency = ranked_df["frequency"]

mil_base = bases_df[["lat", "long"]]

fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=frequency,
                                dissipating=False, max_intensity=100, 
                                point_radius=1)
fig.add_layer(heat_layer)

symbols = gmaps.symbol_layer(mil_base, fill_color='blue', stroke_color='blue')
#symbols
fig.add_layer(symbols)
marker_layer = gmaps.marker_layer(mil_base)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))